## About

*winterreise_rt* is a subset of Schubert Winterreise Dataset(SWD) [1] for real-time lyrics alignment.
The Schubert Winterreise Dataset(SWD) dataset is a collection of resources of Schubert’s song cycle for voice and piano ‘Winterreise’. 
The the song cycle Winterreise D911 (Op. 89) consists of 24 songs composed for solo voice with piano accompaniment.

## Download the latest version of the Schubert Winterreise Dataset (SWD)
First of all, we will download the SWD dataset and revise the file structure to fit the *winterreise_rt* dataset from the scratch.

In [2]:
!wget "https://zenodo.org/record/5139893/files/Schubert_Winterreise_Dataset_v2-0.zip?download=1" -O winterreise.zip
!unzip winterreise -d winterreise
!rm -r winterreise.zip

--2023-09-13 04:53:18--  https://zenodo.org/record/5139893/files/Schubert_Winterreise_Dataset_v2-0.zip?download=1
Resolving zenodo.org (zenodo.org)... 188.185.124.72
Connecting to zenodo.org (zenodo.org)|188.185.124.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 506943733 (483M) [application/octet-stream]
Saving to: ‘winterreise.zip’

winterreise.zip      44%[=======>            ] 214.06M   114KB/s    eta 37m 57s^C
Archive:  winterreise.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
Archive:  winterreise.ZIP
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of thi

In [21]:
from pathlib import Path
import shutil
import os
from tqdm import tqdm
import csv

SWD_PATH = Path("./winterreise")
WINTERREISE_RT_PATH = Path("./winterreise_rt")
AUDIO_DIR = "01_RawData/audio_wav/"
SCORE_DIR = "01_RawData/score_musicxml/"
LYRICS_DIR = "01_RawData/lyrics_txt/"
NOTE_ANN_DIR = "02_Annotations/ann_audio_note/"
AUDIO_KEY_ANN_DIR = "03_ExtraMaterial/"
FILENAME_PREFIX = "Schubert_D911-"
SINGERS = {
    "HU33": "ref",
    "SC06": "target",
}  # singer_name: role
SONG_IDS = ["{:02d}".format(i) for i in range(1, 25)]  # 01 ~ 24

In [24]:
(WINTERREISE_RT_PATH / AUDIO_DIR).mkdir(parents=True)
(WINTERREISE_RT_PATH / SCORE_DIR).mkdir(parents=True)
(WINTERREISE_RT_PATH / LYRICS_DIR).mkdir(parents=True)
(WINTERREISE_RT_PATH / NOTE_ANN_DIR).mkdir(parents=True)
(WINTERREISE_RT_PATH / AUDIO_KEY_ANN_DIR).mkdir(parents=True)

In [25]:
# Copy audio_wav, score_musicxml, lyrics_txt from Schubert_Winterreise_Dataset_v2-0 to winterreise_rt
def copy_audio_wav():
    print("Copying audio_wav...")
    for song_id in tqdm(SONG_IDS):
        for singer, role in SINGERS.items():
            src_audio = SWD_PATH / AUDIO_DIR / f"{FILENAME_PREFIX}{song_id}_{singer}.wav"
            dest_audio = WINTERREISE_RT_PATH / AUDIO_DIR / f"{FILENAME_PREFIX}{song_id}_{role}.wav"
            shutil.copy(src_audio, dest_audio)

def copy_audio_key_ann():
    print("Copying audio key annotation...")
    shutil.copy(
        SWD_PATH / AUDIO_KEY_ANN_DIR / "ann_audio_globalkey-tuning.csv", 
        WINTERREISE_RT_PATH / AUDIO_KEY_ANN_DIR / "ann_audio_globalkey-tuning.csv"
    )

def copy_score_musicxml():
    print("Copying score...")
    for song_id in tqdm(SONG_IDS):
        src_score = SWD_PATH / SCORE_DIR / f"{FILENAME_PREFIX}{song_id}.xml"
        dest_score = WINTERREISE_RT_PATH / SCORE_DIR / f"{FILENAME_PREFIX}{song_id}.xml"
        shutil.copy(src_score, dest_score)

def copy_lyrics_txt():
    print("Copying lyrics...")
    for song_id in tqdm(SONG_IDS):
        src_lyrics = SWD_PATH / LYRICS_DIR / f"{FILENAME_PREFIX}{song_id}.txt"
        dest_lyrics = WINTERREISE_RT_PATH / LYRICS_DIR / f"{FILENAME_PREFIX}{song_id}.txt"
        shutil.copy(src_lyrics, dest_lyrics)

def copy_and_filter_voice_note_ann():
    print("Copying & filtering voice note annotation...")
    for song_id in tqdm(SONG_IDS):
        for singer, role in SINGERS.items():
            annots = []
            source_file = (
                SWD_PATH
                / NOTE_ANN_DIR
                / f"{FILENAME_PREFIX}{song_id}_{singer}.csv"
            )
            with open(source_file.as_posix()) as file:
                reader = csv.DictReader(file, delimiter=";")
                for row in reader:
                    start, pitch, pitchclass, instrument = (
                        row["start"],
                        row["pitch"],
                        row["pitchclass"],
                        row["instrument"],
                    )
                    if instrument != "voice":
                        continue
                    annots.append(
                        {
                            "start": start,
                            "pitch": pitch,
                            "pitchclass": pitchclass,
                            "instrument": instrument,
                        }
                    )
            out_file = WINTERREISE_RT_PATH / NOTE_ANN_DIR / f"ann_{FILENAME_PREFIX}{song_id}_{role}.csv"
            with open(out_file.as_posix(), "w") as file:
                fieldnames = ["start", "pitch", "pitchclass", "instrument"]
                writer = csv.DictWriter(file, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(annots)

def copy_from_orig_dataset():
    copy_audio_wav()
    copy_audio_key_ann()
    copy_score_musicxml()
    copy_lyrics_txt()
    copy_and_filter_voice_note_ann()

copy_from_orig_dataset()

Copying audio_wav...


100%|██████████| 24/24 [00:00<00:00, 263.76it/s]


Copying audio key annotation...
Copying score...


100%|██████████| 24/24 [00:00<00:00, 4075.44it/s]


Copying lyrics...


100%|██████████| 24/24 [00:00<00:00, 3375.92it/s]


Copying & filtering voice note annotation...


100%|██████████| 24/24 [00:00<00:00, 213.42it/s]


In [ ]:
# Transpose the audio files to match the reference audio


In [ ]:
# Filter out the non-voice notes from the note annotation files
def filter_voice_note_ann():
    print("Copying & filtering voice note annotation...")
    for song_id in tqdm(SONG_IDS):
        for singer, role in SINGERS.items():
            annots = []
            source_file = (
                SWD_PATH
                / NOTE_ANN_DIR
                / f"{FILENAME_PREFIX}{song_id}_{singer}.csv"
            )
            with open(source_file.as_posix()) as file:
                reader = csv.DictReader(file, delimiter=";")
                for row in reader:
                    start, pitch, pitchclass, instrument = (
                        row["start"],
                        row["pitch"],
                        row["pitchclass"],
                        row["instrument"],
                    )
                    if instrument != "voice":
                        continue
                    annots.append(
                        {
                            "start": start,
                            "pitch": pitch,
                            "pitchclass": pitchclass,
                            "instrument": instrument,
                        }
                    )
            out_file = WINTERREISE_RT_PATH / NOTE_ANN_DIR / f"ann_{FILENAME_PREFIX}{song_id}_{role}.csv"
            with open(out_file.as_posix(), "w") as file:
                fieldnames = ["start", "pitch", "pitchclass", "instrument"]
                writer = csv.DictWriter(file, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(annots)

## References

[1] C. Weiß, F. Zalkow, V. Arifi-Müller, M. Müller, H. V.Koops, A. Volk, and H. Grohganz, “Schubert Winterreise dataset: A multimodal scenario for music analysis,” ACM Journal on Computing and Cultural Heritage (JOCCH), vol. 14, no. 2, pp. 25:1–18, 2021.